### 平衡二叉树
#### 1. 为什么有平衡二叉树
&emsp;&emsp;大多数BST操作，例如查找，找最大，最小值，插入，删除等操作，基本上消耗O(h)时间，h是BST的高度。对于倾斜的二叉树，这些操作的成本可能会变成O(n)。

&emsp;&emsp;如果我们在每次插入和删除之后确保树的高度保持O(logn)，那么我们就可以保证所有这些操作的O(logn）的上界。而AVL树的高度总是O(logn)，其中n是树中节点的数量。

&emsp;&emsp;所以AVL树很好的解决了二叉搜索树退化成链表的问题，把插入，查找，删除的时间复杂度的最好情况和最坏情况都维持在O(logn)。

#### 2. 平衡二叉树的定义
&emsp;&emsp;AVL树是带有平衡条件的二叉查找树。它要求在AVL树中任何节点的两个子树的高度(高度是指节点到一片树叶的最长路径的长) 最大差别为1。

#### 3. 旋转
&emsp;&emsp;在介绍插入和删除操作之前，先了解以下二叉树的旋转操作。
##### 3.1 左旋
```
      b                          a
    /   \                      /   \
   T1   a            ----->   b    T3
       /  \                  /  \
      T2  T3                T1  T2     
```
&emsp;&emsp;以ab为轴,a为节点进行左边旋转，其中a的左子树变成b的右子树，a成为新的根节点

In [3]:
def left_rotate(b):
    a = b.right
    b.right = a.left
    a.left = b
    # 其中a,b的高度会变化
    b.height=max(get_height(b.right),get_height(b.left))+1
    a.height=max(get_height(a.right),b.height)+1
    return a

##### 3.2 右旋
```
       a                          b
     /   \                      /   \
    b    T3         ----->     T1    a
  /  \                              /  \
 T1  T2                            T2  T3     
```
&emsp;&emsp;以ab为轴，b为节点进行右边旋转，其中b的右子树变成a的左子树，b成为新的根节点

In [4]:
def right_rotate(a):
    b = a.left
    a.left = b.right
    b.right = a
    # 其中a b高度会有变化
    a.height = max(get_height(a.left),get_height(a.right)) + 1
    b.height = max(get_height(b.left),a.height) + 1
    return b

#### 4. 插入
&emsp;&emsp;插入一个节点可能会破坏avl树的平衡，可以通过旋转操作进行修正，插入一个节点后，只有从插入节点到根节点的路径上的节点的平衡可能会改变。需要找出第一个破坏了平衡条件的节点，称之为k，k的两颗子树的高度差为2。

&emsp;&emsp;不平衡有四种情况

    1.LL型调整（k的左子树较高，新节点插入在k的左子树的左子树）
    2.LR型调整（k的左子树较高，新节点插入在k的左子树的右子树）
    3.RL型调整（k的右子树较高，新节点插入在k的右子树的左子树）
    4.RR型调整（k的右子树较高，新节点插入在k的右子树的右子树）
    
&emsp;&emsp;对于LL型调整，可以通过右旋保持平衡二叉树性质；对于RR型调整，可以通过左旋保持平衡二叉树性质。
```
                 LL型调整
                 
           a                          b
         /   \                      /   \
        b    T3         ----->     T1    a
      /  \                        /     /  \
     T1  T2                      x     T2  T3     
    /
   x 
   
   
                 RR型调整
                 
           b                          a
         /    \                      /   \
        T1    a            ----->   b    T3
             /  \                  /  \   \
           T2   T3                T1  T2  x
                 \
                 x
```

In [5]:
def ll(a):
    return right_rotate(a)

def rr(b):
    return left_rotate(b)

&emsp;&emsp;对于LR型调整，就是a的左子树b的高度较高，然后b的右子树c的高度较高，然后节点插入在以c为根节点的子树中，可以通过两次旋转保持平衡二叉树性质，RL型调整类似。
```               
                      LR型调整
                 
           a                          a                       c
         /   \                      /   \                  /     \ 
        b    T4     ----->        c    T4   ------>       b      a  
      /  \                       /  \                   /  \    /  \
     T1   c                     b   T3                 T1  T2  T3   T4
         / \                   / \   \                          \
        T2 T3                 T1 T2  x                           x
            \
             x
             
                      RL型调整
                 
           a                          a                       c
         /   \                      /   \                  /     \ 
        T1    b     ----->        T1     c   ------>       a      b  
            /   \                       /  \             /  \    /  \   
           c    T4                     T2   b          T1  T2  T3   T4
          / \                             /  \                  \
         T2 T3                           T3  T4                  x
             \                             \
              x                             x
             
```

In [8]:
def lr(a):
    a.left = left_rotate(a.left)
    return right_rorate(a)

def rl(a):
    a.right = right_rotate(a.right)
    return left_rotate(a)


def put(self,key):
    if self.root is None:
        self.root = Node(key)
    else:
        self.root = self._put(key,self.root)

def _put(self,key,node):
    if node is None:
        node = Node(key)
    elif key < node.key:
        node.left = self._put(key,node.left)
        if (self.height(node.left) - self.height(node.right)) == 2:
            if key < node.left.key:
                node = ll(node)
            elif key > node.left.key:
                node = lr(node)
    elif key > node.key:
        node.right = self._put(key,node.right)
        if (self.height(node.right) - self.height(node.left)) == 2:
            if key < node.right.key:
                node = rl(node)
            elif key > node.right.key:
                node = rr(node)
    node.height = max(self.height(node.left),self.height(node.right)) + 1
    return node

#### 5. 删除
&emsp;&emsp;删除操作和普通二叉树类似，分析如下：

    1.当前节点为要删除的节点其是叶子节点，直接删除，当前节点的平衡不受影响
    2.当前节点为要删除的节点且只有一个左儿子或者右儿子，用左儿子或者右儿子替代当前节点，当前节点的平衡不受影响。因为是平衡二叉树，所以左子树或者右子树一定是叶子节点，
    3.当前节点为要删除的节点且有左子树和右子树，如果右子树高度较高，则从右子树选取最小节点，将其值赋予给当前节点，然后删除右子树的最小节点；如果左子树高度较高，则从左子树选取最大节点，将其值赋给当前节点，然后删除左子树的最大节点，这样操作当前节点的平衡不会被破坏。
    4.当前节点不是要删除的节点，则对其左子树或者右子树进行递归操作，当前节点的平衡条件可能会被破坏，需要进行平衡操作。

In [11]:
def delete(self,key):
    self.root=self.remove(key,self.root)
def remove(self,key,node):
    if node is None:
        raise KeyError('Error,key not in tree')
    elif key<node.key:
        node.left=self.remove(key,node.left)
        if (self.height(node.right)-self.height(node.left))==2:
            if self.height(node.right.right)>=self.height(node.right.left):
                node=self.singleRightRotate(node)
            else:
                node=self.doubleRightRotate(node)
        node.height=max(self.height(node.left),self.height(node.right))+1
         
             
    elif key>node.key:
        node.right=self.remove(key,node.right)
        if (self.height(node.left)-self.height(node.right))==2:
            if self.height(node.left.left)>=self.height(node.left.right):
                node=self.singleLeftRotate(node)
            else:
                node=self.doubleLeftRotate(node)
        node.height=max(self.height(node.left),self.height(node.right))+1
     
    elif node.left and node.right:
        if node.left.height<=node.right.height:
            minNode=self._findMin(node.right)
            node.key=minNode.key
            node.right=self.remove(node.key,node.right)
        else:
            maxNode=self._findMax(node.left)
            node.key=maxNode.key
            node.left=self.remove(node.key,node.left)
        node.height=max(self.height(node.left),self.height(node.right))+1
    else:
        if node.right:
            node=node.right
        else:
            node=node.left
     
    return node